hihii

GPU预备

In [1]:
import torch
import pandas as pd
import numpy as np
# 检查是否有可用的GPU
if torch.cuda.is_available():
    print(f"CUDA is available. Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available. Using CPU.")

CUDA is available. Number of GPUs: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3060 Laptop GPU


bert test

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

_HF_DEFAULT_ENDPOINT = "https://hf-mirror.com"
# 指定模型名称
model_name = 'tiktoken'

# 读取模型对应的tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 载入模型
model = AutoModel.from_pretrained(model_name)

# 输入文本
input_text = "Here is some text to encode. This is a test."

# 通过tokenizer把文本变成 token_id
input_ids = tokenizer.encode(input_text, add_special_tokens=True)
print(input_ids)

# 转换为Tensor
input_ids = torch.tensor([input_ids])

# 获得BERT的输出
with torch.no_grad():
    output = model(input_ids)

# 获得BERT模型最后一个隐层结果
output_hidden_states = output.last_hidden_state
output_hidden_states.shape

c:\Users\86139\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


数据集预处理

In [ ]:
file_path =r'D:\deeplearning\big project\approach1\lmsys-chat-1m-llama-3.2-1b-instruct-10K-batch100.json'
import os

import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm


if os.path.exists(file_path):
    print("File exists.")
else:
    print("File does not exist.")
df=pd.read_json(file_path)

def categorize_length(length):
    if length < 200:
        return 0
    elif length < 400:
        return 1
    elif length < 600:
        return 2
    elif length < 800:
        return 3
    else:
        return 4

df['length_category'] = df['generated_length'].apply(categorize_length)

df.head()
text=df['prompt'].tolist()
label=df['length_category'].tolist()
trainlength=len(text)*0.7

train_texts=text[:int(trainlength)]
train_labels=label[:int(trainlength)]
test_texts=text[int(trainlength):]
test_labels=label[int(trainlength):]
#加载 BERT Tokenizer
model_name = "bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
class PromptDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        } # 此处会自动生成BERT输入所需要的attention_mask


train_dataset =PromptDataset(train_texts, train_labels, tokenizer)
test_dataset = PromptDataset(test_texts, test_labels, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

File exists.


In [ ]:
max_length = max(len(text) for text in train_texts)
print(f"The maximum string length in train_texts is: {max_length}")

The maximum string length in train_texts is: 2668


In [ ]:
#!for /L %i in (1,1,10) do (nvidia-smi & timeout /t 5 /nobreak > nul)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
import torch.nn as nn
from transformers import AutoModelForSequenceClassification
""" class BERTClassifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(BERTClassifier, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits """
class BERTClassifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(BERTClassifier, self).__init__()
        # 使用基础BERT模型，不带分类头
        self.bert = AutoModel.from_pretrained(model_name)
        # 添加分类层
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        
    def forward(self, input_ids, attention_mask):
        # 获取BERT输出
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # 提取CLS向量表征
        cls_embedding = outputs.last_hidden_state[:,0,:]
        # 通过分类器得到logits
        logits = self.classifier(cls_embedding)
        return logits
model = BERTClassifier(model_name, num_labels=5).to(device)


# **5. 设置优化器和损失函数**
#TODO: 定义优化器和损失函数
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-5)

# **6. 训练 BERT**
EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")

    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        #TODO: 基于后面需要打印的损失，定义训练过程
        outputs=model(input_ids,attention_mask)
        l=loss(outputs,labels)
        l.backward()
        optimizer.step()
        total_loss+=l.item()
        
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_dataloader):.4f}")

    # **7. 评估模型**
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            logits = model(input_ids, attention_mask)
            predictions = torch.argmax(logits, dim=-1)
            #TODO: 基于后面计算acc需要的true_labels和preds，完善下面测试代码
            preds.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())


    acc = accuracy_score(true_labels, preds)
    print(f"Test Accuracy: {acc:.4f}")

Epoch 1:   0%|          | 0/438 [00:00<?, ?it/s]c:\Users\86139\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Epoch 1: 100%|██████████| 438/438 [21:27<00:00,  2.94s/it]


Epoch 1, Loss: 0.9849
Test Accuracy: 0.6737


Epoch 2:   0%|          | 1/438 [00:05<43:08,  5.92s/it]


KeyboardInterrupt: 